In [1]:
import pandas as pd
import time 
start_time = time.time()

In [2]:
# drugs = pd.read_csv('arcos-ca-statewide-itemized.tsv',nrows = 1000000, sep = '\t')
# drugs = pd.read_csv('arcos-ca-statewide-itemized.tsv', sep = '\t')


In [20]:
zip_pop = pd.read_csv('../ZCTA_area.csv')

In [21]:
zip_pop.head(10)

,AFFGEOID10,ZIPcode,Area_ac,Area_sqmi
0,8600000US21914,21914,478.857880,0.748215
1,8600000US01001,1001,7877.861328,12.309158
2,8600000US34736,34736,98956.843750,154.620071
3,8600000US46151,46151,133556.187500,208.681549
4,8600000US48039,48039,15943.142580,24.911160
5,8600000US01521,1521,8370.506836,13.078917
6,8600000US49892,49892,106387.203100,166.229996
7,8600000US70639,70639,44236.843750,69.120071
8,8600000US56755,56755,30904.103520,48.287663
9,8600000US64723,64723,29107.585940,45.480602


# Connecting to S3

In [5]:
from pyspark import SparkContext, SparkConf
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:2.7.4" pyspark-shell'
sc = SparkContext.getOrCreate()

with open("../../../Keys/aws.csv") as f:
    data = f.read().splitlines()[1].split(",")
    access = data[2]
    sec = data[3]

# YOUR AMAZON LOGIN INFORMATION HERE
sc._jsc.hadoopConfiguration().set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', f"{access}")

sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', f"{sec}")

## Exploring the Dataset

In [6]:
# drugs.to_csv('./arcos-ca-statewide-sample.tsv', sep = '\t', index = False)

In [7]:
# drugs = pd.read_csv('./arcos-ca-statewide-sample.tsv', sep = '\t')
# drugs.head()

In [8]:
# Agg Sum - CALC_BASE_WT_IN_GM, QUANTITY
# Ingredient_Name has two options: HYDROCODONE BITARTRATE HEMIPENTAHYDRATE or OXYCODONE HYDROCHLORIDE

## Working on sample of Drug Dataset to make a DataFrame

In [9]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
#from pyspark.sql.functions import concat, col, lit, substring

ss = SparkSession.builder.getOrCreate()

## !!!!! Change this to the S3 Bucket !!!!!!
# drug_rdd = sc.textFile('./arcos-ca-statewide-sample.tsv').map(lambda x: x.split('\t'))
drug_rdd = sc.textFile('s3://data-systems-opioid/arcos-ca-statewide-itemized.tsv',24).map(lambda x: x.split('\t'))


def FloatSafe(value): # In case there are non-float type to be converted.
    try:
        return float(value)
    except ValueError:
        return None

def IntegerSafe(value): # In case there are non-integer type to be converted.
    try:
        return int(value)
    except ValueError:
        return None

# To reduce size, I remove the first 15 col's which are all unusable Identifiers
drug_rdd = drug_rdd.map(lambda x: x[16:25] + x[29:]).persist()

# Columns I removed:
# -------------------------
# 0'REPORTER_DEA_NO',
#  'REPORTER_BUS_ACT',
#  'REPORTER_NAME',
#  'REPORTER_ADDL_CO_INFO',
#  'REPORTER_ADDRESS1',
#  'REPORTER_ADDRESS2',
#  'REPORTER_CITY',
#  'REPORTER_STATE',
#  'REPORTER_ZIP',
#  'REPORTER_COUNTY'
#  'BUYER_DEA_NO',
#  'BUYER_BUS_ACT',
#  'BUYER_NAME',
#  'BUYER_ADDL_CO_INFO',
#  'BUYER_ADDRESS1',
#15'BUYER_ADDRESS2'

# 25 UNIT (0.001% of rows have values)
# 26 Action Indicator
# 27 ORDER_FORM_NO
# 28 CORRECTION_NO

# Takes header row and makes column names
col_names = drug_rdd.first()

# Removes header col
drug_rdd = drug_rdd.filter(lambda x: x != col_names)

# Fixes variable type
drug_rdd = drug_rdd.map(lambda x: [x[0], x[1], x[2], x[3], x[4], IntegerSafe(x[5]), x[6], x[7], FloatSafe(x[8]), FloatSafe(x[9]), IntegerSafe(x[10]), FloatSafe(x[11]), FloatSafe(x[12]), IntegerSafe(x[13]),x[14], x[15], x[16], FloatSafe(x[17]), x[18], x[19], x[20], FloatSafe(x[21])])


In [10]:
print(col_names)

['BUYER_CITY', 'BUYER_STATE', 'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO', 'DRUG_NAME', 'QUANTITY', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM', 'DOSAGE_UNIT', 'TRANSACTION_ID', 'Product_Name', 'Ingredient_Name', 'Measure', 'MME_Conversion_Factor', 'Combined_Labeler_Name', 'Revised_Company_Name', 'Reporter_family', 'dos_str']


In [11]:
# # This cell is just for Type testing
# row_test = drug_rdd.takeSample(1, 1)[0]
# for i in range(len(row_test)):
#     print(i, col_names[i], row_test[i], type(row_test[i]))

In [12]:
# To DataFrame
drug_df = drug_rdd.toDF(col_names)

# Set up for the ZIP-YEAR join
drug_df = drug_df.withColumn('Year', substring('TRANSACTION_DATE', -4,4))
drug_df = drug_df.withColumn('ZIP-YEAR', concat(col("BUYER_ZIP"), lit("-"), col("Year")))
drug_df.select("BUYER_ZIP","Year",'ZIP-YEAR').show(10)

+---------+----+----------+
|BUYER_ZIP|Year|  ZIP-YEAR|
+---------+----+----------+
|    93003|2007|93003-2007|
|    92649|2006|92649-2006|
|    92653|2006|92653-2006|
|    92113|2006|92113-2006|
|    92113|2006|92113-2006|
|    91301|2007|91301-2007|
|    92584|2007|92584-2007|
|    93402|2012|93402-2012|
|    93065|2012|93065-2012|
|    95536|2011|95536-2011|
+---------+----+----------+
only showing top 10 rows



# Population Per Square Mile DataFrame

In [26]:
pop_sq_mile = sc.textFile('../ZCTA_area.csv', 24).map(lambda x: x.split(','))
col_names = pop_sq_mile.take(1)[0]
pop_df = pop_sq_mile.filter(lambda x: x != col_names).toDF(col_names)

In [27]:
pop_df.show(5)

+--------------+-------+-----------+-----------+
|    AFFGEOID10|ZIPcode|    Area_ac|  Area_sqmi|
+--------------+-------+-----------+-----------+
|8600000US21914|  21914|478.8578796|0.748215437|
|8600000US01001|   1001|7877.861328|12.30915833|
|8600000US34736|  34736|98956.84375|154.6200714|
|8600000US46151|  46151|133556.1875|208.6815491|
|8600000US48039|  48039|15943.14258|24.91115952|
+--------------+-------+-----------+-----------+
only showing top 5 rows



# Suicide DataFrame

In [28]:
death_rdd = sc.textFile('s3://data-systems-opioid/CA_suicides.csv',24).map(lambda x: x.split(','))

# Takes header row and makes column names
col_names = death_rdd.first()

# Removes header col
death_rdd = death_rdd.filter(lambda x: x != col_names)

# Fix RDD
death_rdd = death_rdd.map(lambda x: [x[0], x[1], x[2], IntegerSafe(x[3]), IntegerSafe(x[4])])

# To SQL DataFrame
death_df = death_rdd.toDF(col_names)
death_df = death_df.withColumn('SUI_per_thousand', death_df['Count']/death_df['Population_2018'] * 1000)

# Set Up for the ZIP-YEAR merg
death_df = death_df.withColumn('ZIP-YEAR', concat(col("ZIP Code"), lit("-"), col("Year")))

## Joining drug_df and death_df

In [30]:
# Change this join based on what flags we add
drug_agg_df = drug_df.groupBy('ZIP-YEAR').agg(count('BUYER_CITY'), sum('STRENGTH'), sum('QUANTITY'), sum('CALC_BASE_WT_IN_GM'), sum('dos_str'))
# drug_agg_df.show(5)

In [31]:
death_join_df = death_df.select('ZIP-YEAR','Count','SUI_per_thousand')
drug_death_df = drug_agg_df.join(death_join_df, 'ZIP-YEAR', 'left_outer')
drug_death_df.show(1)

Py4JJavaError: An error occurred while calling o330.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 53 in stage 21.0 failed 1 times, most recent failure: Lost task 53.0 in stage 21.0 (TID 97, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
	at or

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60002)
Traceback (most recent call last):
  File "/Users/MatthewKing/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/MatthewKing/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/MatthewKing/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/MatthewKing/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/Users/MatthewKing/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/Users/MatthewKing/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/Users/Matthew

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))